# Unauthorized System Time Modification
Detect scenarios where a potentially unauthorized application or user is modifying the system time.

## Rule Content
```
- title: Unauthorized System Time Modification
  id: faa031b5-21ed-4e02-8881-2591f98d82ed
  status: experimental
  description: Detect scenarios where a potentially unauthorized application or user
    is modifying the system time.
  author: '@neu5ron'
  references:
  - Private Cuckoo Sandbox (from many years ago, no longer have hash, NDA as well)
  - Live environment caused by malware
  date: 2019/02/05
  tags:
  - attack.defense_evasion
  - attack.t1099
  logsource:
    product: windows
    service: security
    definition: 'Requirements: Audit Policy : System > Audit Security State Change,
      Group Policy : Computer Configuration\Windows Settings\Security Settings\Advanced
      Audit Policy Configuration\Audit Policies\System\Audit Security State Change'
    category: null
  detection:
    selection:
      EventID: 4616
    filter1:
      ProcessName: C:\Program Files\VMware\VMware Tools\vmtoolsd.exe
    filter2:
      ProcessName: C:\Windows\System32\VBoxService.exe
    filter3:
      ProcessName: C:\Windows\System32\svchost.exe
      SubjectUserSid: S-1-5-19
    condition: selection and not ( filter1 or filter2 or filter3 )
  falsepositives:
  - HyperV or other virtualization technologies with binary not listed in filter portion
    of detection
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-endpoint-winevent-security-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(event_id:"4616" AND (NOT (((process_path:"C\:\\Program\ Files\\VMware\\VMware\ Tools\\vmtoolsd.exe" OR process_path:"C\:\\Windows\\System32\\VBoxService.exe") OR (process_path:"C\:\\Windows\\System32\\svchost.exe" AND SubjectUserSid:"S\-1\-5\-19")))))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()